##### I want to recapitulate all the steps necessary to reproduce the paper results

##### converting files

##### shell commands

##### first, I had to merge the phase 2 files with the VObs files. then I had to convert.

##### first I had to make the VObs files: I kept only those sites present in phase 2, and removed certain specimens

##### I'll work on compiling all that in a separate file, and right now, will compile the Python steps

## IDing training and validation set

## classifying by PCA

In [ ]:
import allel
from collections import namedtuple
import datetime
import h5py
import ingenos
import itertools
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from matplotlib import collections as mc
import numpy as np
import pandas as pd
import re
import seaborn as sns
from sklearn import model_selection
%matplotlib inline

##### read in data for 2R

In [ ]:
v_2R, g_2R = ingenos.import_data(
    "/afs/crc.nd.edu/group/BesanskyNGS2/inversion_genotyping/merged_p2_and_VObs_2R.h5", "2R")

##### read in data for 2L

In [ ]:
path_2L = "/afs/crc.nd.edu/group/BesanskyNGS/inversion_genotyping/merged_p2_and_VObs_2L.h5"
chrom_2L = "2L"

callset_2L = h5py.File(path_2L, mode='r')[chrom_2L]

v_2L = allel.VariantChunkedTable(callset_2L['variants'], index='POS',
                                names=['POS','REF','ALT','DP','MQ','QD','numalt'])

g_2L = allel.GenotypeChunkedArray(callset_2L['calldata']['GT'])

##### read in metadata

##### note to self: add the metadata files without the PCA or computational karyotypes-- before I ran the below code

In [1]:
md_2L = pd.read_csv("../metadata/all_samples_2L_metadata_080318.csv", sep="\t")
md_2R = pd.read_csv("../metadata/all_samples_2R_metadata_080318.csv", sep="\t")

NameError: name 'pd' is not defined

##### create filters to keep the correct partitions for each inversion

In [ ]:
non_outliers = ((md_2R["country"] != "Kenya") &
                (md_2R["country"] != "Gambia, The") &
                (md_2R["country"] != "Guinea-Bissau")).values

west = (md_2R["country"] != "Kenya").values

j_bool = ((md_2R["country"] != "Kenya") & (md_2R["country"] != "Gambia, The") &\
          (md_2R["country"] != "Guinea-Bissau") & (md_2R["country"] != "France") &\
         (md_2R["species"] == "An. gambiae")).values

col_bool = ( (md_2R["ox_code"].isin(md_2R.loc[non_outliers,"ox_code"])) &\
                 (md_2R["species"] == "An. coluzzii"))

gam_bool = ( (md_2R["ox_code"].isin(md_2R.loc[non_outliers,"ox_code"])) &\
                 (md_2R["species"] == "An. gambiae"))

##### 2La

In [ ]:
coords_2La, model_2La = ingenos.run_pca("2La", v_2L, g_2L)

In [ ]:
fig, ax = plt.subplots(figsize = (8, 8))
sns.despine(ax = ax, offset = 10)
ingenos.plot_pca_coords(coords_2La, model_2La, 0, 1, ax, md_2L,
                region = "2La", label_with = "2La",
               alpha = 0.5, title = "2La")

ax.text( -375, 150, "0", fontsize = 20, weight = 'bold')
ax.text( 0, 150, "1", fontsize = 20, weight = 'bold')
ax.text( 350, 150, "2", fontsize = 20, weight = 'bold')

ax.axvline(x = -200, color = 'k', linewidth = 1)
ax.axvline(x = 200, color = 'k', linewidth = 1)
plt.title("2La", fontsize=20)
ax.xaxis.label.set_size(20)
ax.yaxis.label.set_size(20)
plt.tick_params(axis='both', which='major', labelsize=15)

legend = ax.legend(title = "Cytogenetic\n 2La", fontsize=15)
legend.get_title().set_fontsize('15')

#plt.savefig(ingenos.make_date_stamp("../data/results/2La_karyotyping_PCA"));

In [ ]:
PCA_2La = []

PC1 = coords_2La[:, 0]
PC2 = coords_2La[:, 1]

assert len(PC1) == len(PC2), "Components are somehow different lengths"

for i in range(len(PC1)):
    
    if PC1[i] <= -200:
        
        PCA_2La.append(0)
        
    elif PC1[i] > -200 and PC1[i] <= 200:
        
        PCA_2La.append(1)
        
    else:
        
        PCA_2La.append(2)

##### 2Rj

In [ ]:
coords_2Rj, model_2Rj = ingenos.run_pca("2Rj", v_2R, g_2R, samples_bool=j_bool)

In [ ]:
fig, ax = plt.subplots(figsize = (8, 8))
sns.despine(ax = ax, offset = 10)
ingenos.plot_pca_coords(coords_2Rj, model_2Rj, 0, 1, ax, md_2R[j_bool],
                region = "2Rj", label_with = "2Rj",
               alpha = 0.5, title = "2Rj")

ax.text( -50, 100, "0", fontsize = 20, weight = 'bold')
ax.text( 185, 100, "1", fontsize = 20, weight = 'bold')
ax.text( 275, 100, "2", fontsize = 20, weight = 'bold')

ax.axvline(x = 150, color = 'k', linewidth = 1)
ax.axvline(x = 225, color = 'k', linewidth = 1)

plt.title("2Rj", fontsize=20)
ax.xaxis.label.set_size(20)
ax.yaxis.label.set_size(20)
plt.tick_params(axis='both', which='major', labelsize=15)

legend = ax.legend(title = "Cytogenetic\n 2Rj", fontsize=15)
legend.get_title().set_fontsize('15')

#plt.savefig(ingenos.make_date_stamp("../data/results/2Rj_karyotyping_PCA"));

In [ ]:
PCA_2Rj = []

PC1 = coords_2Rj[:, 0]
PC2 = coords_2Rj[:, 1]

assert len(PC1) == len(PC2), "Components are somehow different lengths"

for i in range(len(PC1)):
    
    if PC1[i] <= 150:
        
        PCA_2Rj.append(0)
        
    elif PC1[i] > 150 and PC1[i] <= 225:
        
        PCA_2Rj.append(1)
        
    else:
        
        PCA_2Rj.append(2)

##### 2Rb

In [ ]:
coords_2Rb, model_2Rb = ingenos.run_pca("2Rb", v_2R, g_2R, samples_bool = west)

In [ ]:
fig, ax = plt.subplots(figsize = (8, 8))
sns.despine(ax = ax, offset = 10)
ingenos.plot_pca_coords(coords_2Rb, model_2Rb, 0, 1, ax, md_2R[west],
                region = "2Rb", label_with = "2Rb",
               alpha = 0.5, title = "2Rb")

ax.text( -150, 75, "0", fontsize = 20, weight = 'bold')
ax.text( 80, 75, "1", fontsize = 20, weight = 'bold')
ax.text( 210, 75, "2", fontsize = 20, weight = 'bold')
ax.text( -175, -85, "0", fontsize = 20, weight = 'bold')
ax.text( 0, -85, "1", fontsize = 20, weight = 'bold')
ax.text( 175, -85, "2", fontsize = 20, weight = 'bold')

ax.axhline(y = -60, color = 'k', linewidth = 1)
ymin, ymax = ax.get_ylim()
plt.vlines(-100, ymin, -60, color='k', linewidth = 1)
plt.vlines(100, ymin, -60, color='k', linewidth = 1)
plt.vlines(0, ymax, -60, color='k', linewidth = 1)
plt.vlines(150, ymax, -60, color='k', linewidth = 1)

ax.set_ylim(bottom=ymin, top=ymax)

plt.title("2Rb", fontsize=20)
ax.xaxis.label.set_size(20)
ax.yaxis.label.set_size(20)
plt.tick_params(axis='both', which='major', labelsize=15)

legend = ax.legend(title = "Cytogenetic\n 2Rb", fontsize=15)
legend.get_title().set_fontsize('15')

#plt.savefig(ingenos.make_date_stamp("../data/results/2Rb_karyotyping_PCA"));

In [ ]:
PCA_2Rb = []

PC1 = coords_2Rb[:, 0]
PC2 = coords_2Rb[:, 1]

assert len(PC1) == len(PC2), "Components are somehow different lengths"

for i in range(len(PC1)):
    
    if PC2[i] <= -60:
        
        if PC1[i] <= -100:
            
            PCA_2Rb.append(0)
            
        elif PC1[i] > -100 and PC1[i] <= 100:
            
            PCA_2Rb.append(1)
            
        else:
            
            PCA_2Rb.append(2)
            
    else:
        
        if PC1[i] <= 0:
            
            PCA_2Rb.append(0)
            
        elif PC1[i] > 0 and PC1[i] <= 150:
            
            PCA_2Rb.append(1)
        
        else:
            
            PCA_2Rb.append(2)

##### 2Rc

In [ ]:
coords_2Rc, model_2Rc = ingenos.run_pca("2Rc", v_2R, g_2R, samples_bool=non_outliers)

In [ ]:
fig, ax = plt.subplots(figsize = (8, 8))
sns.despine(ax = ax, offset = 10)
ingenos.plot_pca_coords(coords_2Rc, model_2Rc, 0, 1, ax, md_2R[non_outliers],
                region = "2Rc", label_with = "2Rc",
               alpha = 0.5, title = "2Rc")

ax.text( -50, 25, "0", fontsize = 20, weight = 'bold')
ax.text( -50, -60, "1", fontsize = 20, weight = 'bold')
ax.text( -50, -175, "2", fontsize = 20, weight = 'bold')

ax.axhline(y = -25, color = 'k', linewidth = 1)
ax.axhline(y = -75, color = 'k', linewidth = 1)

plt.title("2Rc", fontsize=20)
ax.xaxis.label.set_size(20)
ax.yaxis.label.set_size(20)
plt.tick_params(axis='both', which='major', labelsize=15)

legend = ax.legend(title = "Cytogenetic\n 2Rc", fontsize=15)
legend.get_title().set_fontsize('15')

#plt.savefig(ingenos.make_date_stamp("../data/results/2Rc_karyotyping_PCA"));

In [ ]:
PCA_2Rc = []

PC1 = coords_2Rc[:, 0]
PC2 = coords_2Rc[:, 1]

assert len(PC1) == len(PC2), "Components are somehow different lengths"

for i in range(len(PC1)):
    
    if PC2[i] > -25:
        
        PCA_2Rc.append(0)
        
    elif PC2[i] <= -25 and PC2[i] > -75:
        
        PCA_2Rc.append(1)
        
    else:
        
        PCA_2Rc.append(2)

##### 2Rd

##### for 2Rd, we excluded the distal part of the inversion from the PCA to get a signal less influenced by the overlapping 2Ru inversion

In [ ]:
d_sites_expression = '( (POS > 41000000) & (POS < 42375004) )'

d_sites_bool = v_2R.eval(d_sites_expression)

np.sum(d_sites_bool)

In [ ]:
two_Rd_alt_alleles, two_Rd_which_alleles = ingenos.filter_and_convert_genotypes(g_2R,
                                            sites_boolean = d_sites_bool,
                                            samples_boolean = gam_bool,
                                            min_count = 3,
                                            variance_threshold = 0.03)

In [ ]:
coords_2Rd, model_2Rd = allel.stats.pca(two_Rd_alt_alleles)

In [ ]:
fig, ax = plt.subplots(figsize = (8, 8))
sns.despine(ax = ax, offset = 10)
ingenos.plot_pca_coords(coords_2Rd, model_2Rd, 0, 1, ax, md_2R[j_bool],
                region = "2Rd", label_with = "2Rd",
               alpha = 0.5, title = "2Rd")

ax.text( 150, 10, "0", fontsize = 20, weight = 'bold')
ax.text( 150, -100, "1", fontsize = 20, weight = 'bold')
ax.text( 150, -200, "2", fontsize = 20, weight = 'bold')

ax.axhline(y = -50, color = 'k', linewidth = 1)
ax.axhline(y = -150, color = 'k', linewidth = 1)

plt.title("2Rd", fontsize=20)
ax.xaxis.label.set_size(20)
ax.yaxis.label.set_size(20)
plt.tick_params(axis='both', which='major', labelsize=15)

legend = ax.legend(title = "Cytogenetic\n 2Rd", fontsize=15)
legend.get_title().set_fontsize('15')

#plt.savefig(ingenos.make_date_stamp("../data/results/2Rd_karyotyping_PCA"));

In [ ]:
PCA_2Rd = []

PC1 = coords_2Rd[:, 0]
PC2 = coords_2Rd[:, 1]

assert len(PC1) == len(PC2), "Components are somehow different lengths"

for i in range(len(PC1)):
    
    if PC2[i] >= -50:
        
        PCA_2Rd.append(0)
        
    elif PC2[i] < -50 and PC2[i] > -150:
        
        PCA_2Rd.append(1)
        
    else:
        
        PCA_2Rd.append(2)

##### 2Ru

In [ ]:
coords_2Ru, model_2Ru = ingenos.run_pca("2Ru", v_2R, g_2R, samples_bool=non_outliers)

In [ ]:
fig, ax = plt.subplots(figsize = (8, 8))
sns.despine(ax = ax, offset = 10)
ingenos.plot_pca_coords(coords_2Ru, model_2Ru, 0, 1, ax, md_2R[non_outliers],
                region = "2Ru", label_with = "2Ru",
               alpha = 0.5, title = "2Ru")

ax.text( -40, 0, "0", fontsize = 20, weight = 'bold')
ax.text( -40, 100, "1", fontsize = 20, weight = 'bold')
ax.text( -40, 200, "2", fontsize = 20, weight = 'bold')

ax.axhline(y = 150, color = 'k', linewidth = 1)
ax.axhline(y = 50, color = 'k', linewidth = 1)

plt.title("2Ru", fontsize=20)
ax.xaxis.label.set_size(20)
ax.yaxis.label.set_size(20)
plt.tick_params(axis='both', which='major', labelsize=15)

legend = ax.legend(title = "Cytogenetic\n 2Ru", fontsize=15)
legend.get_title().set_fontsize('15')

#plt.savefig(ingenos.make_date_stamp("../data/results/2Ru_karyotyping_PCA"));

In [ ]:
PCA_2Ru = []

PC1 = coords_2Ru[:, 0]
PC2 = coords_2Ru[:, 1]

assert len(PC1) == len(PC2), "Components are somehow different lengths"

for i in range(len(PC1)):
    
    if PC2[i] >= 150:
        
        PCA_2Ru.append(2)
        
    elif PC2[i] < 150 and PC2[i] > 50:
        
        PCA_2Ru.append(1)
        
    else:
        
        PCA_2Ru.append(0)

##### merge these in to the metadata

In [ ]:
md_2L["new_PCA_2La"] = pd.Series(PCA_2La).values

md_2R.loc[(md_2R["country"] != "Kenya"),"new_PCA_2Rb"] = pd.Series(PCA_2Rb).values

md_2R.loc[non_outliers,"new_PCA_2Rc"] = pd.Series(PCA_2Rc).values
md_2R.loc[non_outliers,"new_PCA_2Ru"] = pd.Series(PCA_2Ru).values

md_2R.loc[j_bool,"new_PCA_2Rj"] = pd.Series(PCA_2Rj).values
md_2R.loc[j_bool,"new_PCA_2Rd"] = pd.Series(PCA_2Rd).values

md_2L.fillna(value="None", inplace=True)
md_2R.fillna(value="None", inplace=True)

## IDing tag SNPs

##### read in the identifications of each specimen to either the training or the validation set

In [ ]:
splits = np.load("../metadata/comp_karyo_splits/splits.npy").flat[0]

splits_d = np.load("../metadata/comp_karyo_splits/2Rdj_splits.npy",
                allow_pickle=True).flat[0]

splits["2Rd"] = splits_d["2Rd"]

##### include the code for generating one's own splits if desired

##### subset to specimens that could be called for PCA

In [ ]:
md_2La = pd.DataFrame(md_2L)
md_2Rj = md_2R.loc[md_2R["new_PCA_2Rj"] != "None",:]
md_2Rb = md_2R.loc[md_2R["new_PCA_2Rb"] != "None",:]
md_2Rc = md_2R.loc[md_2R["new_PCA_2Rc"] != "None",:]
md_2Rd = md_2R.loc[md_2R["new_PCA_2Rd"] != "None",:]
md_2Ru = md_2R.loc[md_2R["new_PCA_2Ru"] != "None",:]

In [ ]:
own_splits = {"2Rb" : {}, "2Rc" : {}, "2La" : {}, "2Rd" : {}, "2Ru" : {}, "2Rj" : {}}

In [ ]:
own_splits["2La"]["train"], own_splits["2La"]["test"] =\
model_selection.train_test_split(md_2La["ox_code"].values, train_size=0.75, test_size=0.25,
                                 stratify=md_2La["new_PCA_2La"].values)

own_splits["2Rj"]["train"], own_splits["2Rj"]["test"] =\
model_selection.train_test_split(md_2Rj["ox_code"].values, train_size=0.75, test_size=0.25,
                                 stratify=md_2Rj["new_PCA_2Rj"].values)

own_splits["2Rb"]["train"], own_splits["2Rb"]["test"] =\
model_selection.train_test_split(md_2Rb["ox_code"].values, train_size=0.75, test_size=0.25,
                                 stratify=md_2Rb["new_PCA_2Rb"].values)

own_splits["2Rc"]["train"], own_splits["2Rc"]["test"] =\
model_selection.train_test_split(md_2Rc["ox_code"].values, train_size=0.75, test_size=0.25,
                                 stratify=md_2Rc["new_PCA_2Rc"].values)

own_splits["2Rd"]["train"], own_splits["2Rd"]["test"] =\
model_selection.train_test_split(md_2Rd["ox_code"].values, train_size=0.75, test_size=0.25,
                                 stratify=md_2Rd["new_PCA_2Rd"].values)

own_splits["2Ru"]["train"], own_splits["2Ru"]["test"] =\
model_selection.train_test_split(md_2Ru["ox_code"].values, train_size=0.75, test_size=0.25,
                                 stratify=md_2Ru["new_PCA_2Ru"].values)

##### mask low-quality genotypes. We don't do this for PCA, because the results depend on thousands of SNPs. When identifying tag SNPs, however, individual SNPs are the focus and we need to exclude those of low quality.

In [ ]:
merged_2R = h5py.File(
    "/afs/crc.nd.edu/group/BesanskyNGS2/inversion_genotyping/merged_p2_and_VObs_2R.h5", 
    mode="r")

gq_2R = merged_2R["2R"]['calldata']['GQ'][:]

g_2R.mask = gq_2R < 20

In [ ]:
gq_2L = callset_2L['calldata']['GQ'][:]

g_2L.mask = gq_2L < 20

##### 2La

In [ ]:
sites_2La = ingenos.construct_filter_expression("2La", ingenos.inversionDict,
                                                buffer=0, whole_inversion=True)

filter_2La = v_2L.eval(sites_2La)

np.sum(filter_2La)

In [ ]:
g_2La = g_2L.subset(sel1 = md_2L["ox_code"].isin(splits["2La"]["train"]).values)

md_2La = md_2L.loc[md_2L["ox_code"].isin(splits["2La"]["train"]),:]

In [ ]:
a_dict = {}

for i in range(10):
    print(i)
    
    train, test =\
    model_selection.train_test_split(md_2La["ox_code"].values, train_size=0.75, test_size=0.25,
                                 stratify=md_2La["new_PCA_2La"].values)
    
    train_bool = md_2La["ox_code"].isin(train).values
    test_bool = md_2La["ox_code"].isin(test).values
    
    a = ingenos.run_concordance_calculation("2La", v_2L[:], g_2La, 
                                md_2La.loc[train_bool, "new_PCA_2La"].map(float).values,
                                        variance_threshold = 0.05, sites_bool = filter_2La,
                                samples_bool = train_bool)
    
    top = a.loc[((a["called_0"] > 0.9) & (a["called_1"] > 0.9) & (a["called_2"] > 0.9) &\
       (a["min"] > 0.995)),"position"].values
    
    site_indices = np.array([np.where(v_2L["POS"] == site)[0][0] for site in top])

    alts = g_2La.subset(sel0 = site_indices, sel1 = test_bool).to_n_alt()
    
    assigned = []

    for specimen in np.mean(alts, axis=0):
        
        if specimen <= 0.66:
        
            assigned.append(0)
        
        elif specimen > 0.66 and specimen <= 1.33:

            assigned.append(1)

        else:

            assigned.append(2)
            
    md_2La_test = pd.DataFrame(md_2La.loc[test_bool,:])
    
    md_2La_test["assigned"] = pd.Series(assigned).values
    
    mismatches =\
    np.sum(md_2La_test["assigned"] != md_2La_test["new_PCA_2La"])
    
    a_dict[i] = (train, test, a, top, mismatches)

##### average the ten iterations

In [ ]:
a_compiled = []

for i in range(10):
    
    a_compiled.extend(a_dict[i][2]["position"].values)

In [ ]:
a_average = pd.DataFrame({"position" : sorted(set(a_compiled)),
                          "ref" : np.nan,
                          "alt" : np.nan,
                         "score_0" : np.nan, 
                          "score_1" : np.nan, 
                          "score_2" : np.nan,
                         "overall_score" : np.nan, 
                          "called_0" : np.nan, 
                          "called_1" : np.nan,
                         "called_2" : np.nan, 
                          "overall_called" : np.nan, 
                          "min" : np.nan})\
[["position","ref","alt","score_0","score_1","score_2","overall_score","called_0","called_1",
 "called_2","overall_called","min"]]

In [ ]:
for pos in sorted(set(a_compiled)):
    
    rows = {}
    
    refs = []
    
    alts = []

    for i in range(10):

        row = a_dict[i][2][a_dict[i][2]["position"] == pos]

        if len(row) > 0:
            
            refs.append(row["ref"].values[0])
            
            alts.append(row["alt"].values[0])

            rows[i] = row.drop(["ref","alt"], axis=1).reset_index(drop=True)
            
    if not len(set(refs)) == 1 and len(set(alts)) == 1:
        
        raise ValueError(pos, rows)
        
    ref = refs[0]
    
    alt = alts[0]

    mean_row = np.sum(rows[i] for i in rows.keys()) / len(rows.keys())
    
    mean_row["count"] = len(rows.keys())
    
    mean_row["ref"] = ref
    
    mean_row["alt"] = alt
    
    assert mean_row["position"].values[0] == pos, row
        
    a_average = a_average.append(mean_row)

##### 2Rj

In [ ]:
sites_2Rj = ingenos.construct_filter_expression("2Rj", ingenos.inversionDict,
                                                buffer=0, whole_inversion=True)

filter_2Rj = v_2R.eval(sites_2Rj)

np.sum(filter_2Rj)

In [ ]:
g_2Rj = g_2R.subset(sel1 = md_2R["ox_code"].isin(splits["2Rj"]["train"]).values)

md_2Rj = md_2R.loc[md_2R["ox_code"].isin(splits["2Rj"]["train"]),:]

In [ ]:
j_dict = {}

for i in range(10):
    print(i)
    
    train, test =\
    model_selection.train_test_split(md_2Rj["ox_code"].values, train_size=0.75, test_size=0.25,
                                 stratify=md_2Rj["new_PCA_2Rj"].values)
    
    train_bool = md_2Rj["ox_code"].isin(train).values
    test_bool = md_2Rj["ox_code"].isin(test).values
    
    j = ingenos.run_concordance_calculation("2Rj", v_2R[:], g_2Rj, 
                                md_2Rj.loc[train_bool, "new_PCA_2Rj"].map(float).values,
                                        variance_threshold = 0.05, sites_bool = filter_2Rj,
                                samples_bool = train_bool)
    
    top = j.loc[((j["called_0"] > 0.9) & (j["called_1"] > 0.9) & (j["called_2"] > 0.9) &\
       (j["min"] > 0.8)),"position"].values
    
    site_indices = np.array([np.where(v_2R["POS"] == site)[0][0] for site in top])

    alts = g_2Rj.subset(sel0 = site_indices, sel1 = test_bool).to_n_alt()
    
    assigned = []

    for specimen in np.mean(alts, axis=0):
        
        if specimen <= 0.66:
        
            assigned.append(0)
        
        elif specimen > 0.66 and specimen <= 1.33:

            assigned.append(1)

        else:

            assigned.append(2)
            
    md_2Rj_test = pd.DataFrame(md_2Rj.loc[test_bool,:])
    
    md_2Rj_test["assigned"] = pd.Series(assigned).values
    
    mismatches =\
    np.sum(md_2Rj_test["assigned"].map(float).map(str) != md_2Rj_test["new_PCA_2Rj"])
    
    j_dict[i] = (train, test, j, top, mismatches)

In [ ]:
j_compiled = []

for i in range(10):
    
    j_compiled.extend(j_dict[i][2]["position"].values)

In [ ]:
j_average = pd.DataFrame({"position" : sorted(set(j_compiled)),
                          "ref" : np.nan,
                          "alt" : np.nan,
                         "score_0" : np.nan, 
                          "score_1" : np.nan, 
                          "score_2" : np.nan,
                         "overall_score" : np.nan, 
                          "called_0" : np.nan, 
                          "called_1" : np.nan,
                         "called_2" : np.nan, 
                          "overall_called" : np.nan, 
                          "min" : np.nan})\
[["position","ref","alt","score_0","score_1","score_2","overall_score","called_0","called_1",
 "called_2","overall_called","min"]]

In [ ]:
for pos in sorted(set(j_compiled)):
    
    rows = {}
    
    refs = []
    
    alts = []

    for i in range(10):

        row = j_dict[i][2][j_dict[i][2]["position"] == pos]

        if len(row) > 0:
            
            refs.append(row["ref"].values[0])
            
            alts.append(row["alt"].values[0])

            rows[i] = row.drop(["ref","alt"], axis=1).reset_index(drop=True)
            
    if not len(set(refs)) == 1 and len(set(alts)) == 1:
        
        raise ValueError(pos, rows)
        
    ref = refs[0]
    
    alt = alts[0]

    mean_row = np.sum(rows[i] for i in rows.keys()) / len(rows.keys())
    
    mean_row["count"] = len(rows.keys())
    
    mean_row["ref"] = ref
    
    mean_row["alt"] = alt
    
    assert mean_row["position"].values[0] == pos, row
        
    j_average = j_average.append(mean_row)

##### 2Rb

In [ ]:
sites_2Rb = ingenos.construct_filter_expression("2Rb", ingenos.inversionDict,
                                                buffer=0, whole_inversion=True)

filter_2Rb = v_2R.eval(sites_2Rb)

np.sum(filter_2Rb)

In [ ]:
g_2Rb = g_2R.subset(sel1 = md_2R["ox_code"].isin(splits["2Rb"]["train"]).values)

md_2Rb = md_2R.loc[md_2R["ox_code"].isin(splits["2Rb"]["train"]),:]

In [ ]:
b_dict = {}

for i in range(10):
    print(i)
    
    train, test =\
    model_selection.train_test_split(md_2Rb["ox_code"].values, train_size=0.75, test_size=0.25,
                                 stratify=md_2Rb["new_PCA_2Rb"].values)
    
    train_bool = md_2Rb["ox_code"].isin(train).values
    test_bool = md_2Rb["ox_code"].isin(test).values
    
    b = ingenos.run_concordance_calculation("2Rb", v_2R[:], g_2Rb, 
                                md_2Rb.loc[train_bool, "new_PCA_2Rb"].map(float).values,
                                        variance_threshold = 0.05, sites_bool = filter_2Rb,
                                samples_bool = train_bool)
    
    top = b.loc[((b["called_0"] > 0.9) & (b["called_1"] > 0.9) & (b["called_2"] > 0.9) &\
       (b["min"] > 0.8)),"position"].values
    
    site_indices = np.array([np.where(v_2R["POS"] == site)[0][0] for site in top])

    alts = g_2Rb.subset(sel0 = site_indices, sel1 = test_bool).to_n_alt()
    
    assigned = []

    for specimen in np.mean(alts, axis=0):
        
        if specimen <= 0.66:
        
            assigned.append(0)
        
        elif specimen > 0.66 and specimen <= 1.33:

            assigned.append(1)

        else:

            assigned.append(2)
            
    md_2Rb_test = pd.DataFrame(md_2Rb.loc[test_bool,:])
    
    md_2Rb_test["assigned"] = pd.Series(assigned).values
    
    mismatches =\
    np.sum(md_2Rb_test["assigned"].map(float).map(str) != md_2Rb_test["new_PCA_2Rb"])
    
    b_dict[i] = (train, test, b, top, mismatches)

In [ ]:
b_compiled = []

for i in range(10):
    
    b_compiled.extend(b_dict[i][2]["position"].values)

In [ ]:
b_average = pd.DataFrame({"position" : sorted(set(b_compiled)),
                          "ref" : np.nan,
                          "alt" : np.nan,
                         "score_0" : np.nan, 
                          "score_1" : np.nan, 
                          "score_2" : np.nan,
                         "overall_score" : np.nan, 
                          "called_0" : np.nan, 
                          "called_1" : np.nan,
                         "called_2" : np.nan, 
                          "overall_called" : np.nan, 
                          "min" : np.nan})\
[["position","ref","alt","score_0","score_1","score_2","overall_score","called_0","called_1",
 "called_2","overall_called","min"]]

In [ ]:
for pos in sorted(set(b_compiled)):
    
    rows = {}
    
    refs = []
    
    alts = []

    for i in range(10):

        row = b_dict[i][2][b_dict[i][2]["position"] == pos]

        if len(row) > 0:
            
            refs.append(row["ref"].values[0])
            
            alts.append(row["alt"].values[0])

            rows[i] = row.drop(["ref","alt"], axis=1).reset_index(drop=True)
            
    if not len(set(refs)) == 1 and len(set(alts)) == 1:
        
        raise ValueError(pos, rows)
        
    ref = refs[0]
    
    alt = alts[0]

    mean_row = np.sum(rows[i] for i in rows.keys()) / len(rows.keys())
    
    mean_row["count"] = len(rows.keys())
    
    mean_row["ref"] = ref
    
    mean_row["alt"] = alt
    
    assert mean_row["position"].values[0] == pos, row
        
    b_average = b_average.append(mean_row)

##### 2Rc

##### we identified concordant SNPs separetely in coluzzii and gambiae in 2Rc. in addition, we dropped specimens that carried 2Ru in gambiae from the training set.

In [ ]:
sites_2Rc = ingenos.construct_filter_expression("2Rc", ingenos.inversionDict,
                                                buffer=0, whole_inversion=True)

filter_2Rc = v_2R.eval(sites_2Rc)

np.sum(filter_2Rc)

In [ ]:
g_2Rc = g_2R.subset(sel1 = md_2R["ox_code"].isin(splits["2Rc"]["train"]).values)

md_2Rc = md_2R.loc[md_2R["ox_code"].isin(splits["2Rc"]["train"]),:]

##### coluzzii

In [ ]:
col_bool = (md_2Rc["species"] == "An. coluzzii").values

col_c = g_2Rc.subset(sel1 = col_bool)

col_md = md_2Rc.loc[col_bool,:]

In [ ]:
col_c_dict = {}

for i in range(10):
    print(i)
    
    train, test =\
    model_selection.train_test_split(col_md["ox_code"].values, train_size=0.75, test_size=0.25,
                                 stratify=col_md["new_PCA_2Rc"].values)
    
    train_bool = col_md["ox_code"].isin(train).values
    test_bool = col_md["ox_code"].isin(test).values
    
    col_c_conc = ingenos.run_concordance_calculation("2Rc", v_2R[:], col_c, 
                                col_md.loc[train_bool, "new_PCA_2Rc"].map(int).values,
                                        variance_threshold = 0.05, sites_bool = filter_2Rc,
                                samples_bool = train_bool)
    
    top = col_c_conc.loc[((col_c_conc["called_0"] > 0.9) &\
                     (col_c_conc["called_1"] > 0.9) &\
                     (col_c_conc["called_2"] > 0.9) &\
       (col_c_conc["min"] > 0.8)),"position"].values
    
    site_indices = np.array([np.where(v_2R["POS"] == site)[0][0] for site in top])

    alts = col_c.subset(sel0 = site_indices, sel1 = test_bool).to_n_alt()
    
    assigned = []

    for specimen in np.mean(alts, axis=0):
        
        if specimen <= 0.66:
        
            assigned.append(0)
        
        elif specimen > 0.66 and specimen <= 1.33:

            assigned.append(1)

        else:

            assigned.append(2)
            
    col_md_test = pd.DataFrame(col_md.loc[test_bool,:])
    
    col_md_test["assigned"] = pd.Series(assigned).values
    
    mismatches =\
    np.sum(col_md_test["assigned"].map(str) != col_md_test["new_PCA_2Rc"])
    
    col_c_dict[i] = (train, test, col_c_conc, top, mismatches)

In [ ]:
col_c_compiled = []

for i in range(10):
    
    col_c_compiled.extend(col_c_dict[i][2]["position"].values)

In [ ]:
col_c_average = pd.DataFrame({"position" : sorted(set(col_c_compiled)),
                          "ref" : np.nan,
                          "alt" : np.nan,
                         "score_0" : np.nan, 
                          "score_1" : np.nan, 
                          "score_2" : np.nan,
                         "overall_score" : np.nan, 
                          "called_0" : np.nan, 
                          "called_1" : np.nan,
                         "called_2" : np.nan, 
                          "overall_called" : np.nan, 
                          "min" : np.nan})\
[["position","ref","alt","score_0","score_1","score_2","overall_score","called_0","called_1",
 "called_2","overall_called","min"]]

In [ ]:
for pos in sorted(set(compiled)):
    
    rows = {}
    
    refs = []
    
    alts = []

    for i in range(10):

        row = col_c_dict[i][2][col_c_dict[i][2]["position"] == pos]

        if len(row) > 0:
            
            refs.append(row["ref"].values[0])
            
            alts.append(row["alt"].values[0])

            rows[i] = row.drop(["ref","alt"], axis=1).reset_index(drop=True)
            
    if not len(set(refs)) == 1 and len(set(alts)) == 1:
        
        raise ValueError(pos, rows)
        
    ref = refs[0]
    
    alt = alts[0]

    mean_row = np.sum(rows[i] for i in rows.keys()) / len(rows.keys())
    
    mean_row["count"] = len(rows.keys())
    
    mean_row["ref"] = ref
    
    mean_row["alt"] = alt
    
    assert mean_row["position"].values[0] == pos, row
    
    #print(pos, mean_row)
    
    col_c_average = col_c_average.append(mean_row)

##### gambiae

In [ ]:
gam_trim_bool = ((md_2Rc["species"] == "An. gambiae") &\
                 (md_2Rc["new_PCA_2Rj"] != "2.0") &\
                 (md_2Rc["ox_code"] != "AZ0267-C") &\
                (md_2Rc["ox_code"] != "AV0043-C")).values

gam_c = g_2Rc.subset(sel1 = gam_trim_bool)

gam_md = md_2Rc.loc[gam_trim_bool,:]

In [ ]:
gam_c_dict = {}

for i in range(10):
    print(i)
    
    train, test =\
    model_selection.train_test_split(gam_md["ox_code"].values, train_size=0.75, test_size=0.25,
                                 stratify=gam_md["new_PCA_2Rc"].values)
    
    train_bool = gam_md["ox_code"].isin(train).values
    test_bool = gam_md["ox_code"].isin(test).values
    
    gam_c_conc = ingenos.run_concordance_calculation("2Rc", v_2R[:], gam_c, 
                                gam_md.loc[train_bool, "new_PCA_2Rc"].map(float).values,
                                        variance_threshold = 0.05, sites_bool = filter_2Rc,
                                samples_bool = train_bool)
    
    top = gam_c_conc.loc[((gam_c_conc["called_0"] > 0.9) &\
                     (gam_c_conc["called_1"] > 0.9) &\
                     (gam_c_conc["called_2"] > 0.9) &\
       (gam_c_conc["min"] > 0.8)),"position"].values
    
    site_indices = np.array([np.where(v_2R["POS"] == site)[0][0] for site in top])

    alts = gam_c.subset(sel0 = site_indices, sel1 = test_bool).to_n_alt()
    
    assigned = []

    for specimen in np.mean(alts, axis=0):
        
        if specimen <= 0.66:
        
            assigned.append(0)
        
        elif specimen > 0.66 and specimen <= 1.33:

            assigned.append(1)

        else:

            assigned.append(2)
            
    gam_md_test = pd.DataFrame(gam_md.loc[test_bool,:])
    
    gam_md_test["assigned"] = pd.Series(assigned).values
    
    mismatches =\
    np.sum(gam_md_test["assigned"].map(str) != gam_md_test["new_PCA_2Rc"])
    
    gam_c_dict[i] = (train, test, gam_c_conc, top, mismatches)

In [ ]:
gam_c_compiled = []

for i in range(10):
    
    gam_c_compiled.extend(gam_c_dict[i][2]["position"].values)

In [ ]:
gam_c_average = pd.DataFrame({"position" : sorted(set(gam_c_compiled)),
                          "ref" : np.nan,
                          "alt" : np.nan,
                         "score_0" : np.nan, 
                          "score_1" : np.nan, 
                          "score_2" : np.nan,
                         "overall_score" : np.nan, 
                          "called_0" : np.nan, 
                          "called_1" : np.nan,
                         "called_2" : np.nan, 
                          "overall_called" : np.nan, 
                          "min" : np.nan})\
[["position","ref","alt","score_0","score_1","score_2","overall_score","called_0","called_1",
 "called_2","overall_called","min"]]

In [ ]:
for pos in sorted(set(gam_c_compiled)):
    
    rows = {}
    
    refs = []
    
    alts = []

    for i in range(10):

        row = gam_c_dict[i][2][gam_c_dict[i][2]["position"] == pos]

        if len(row) > 0:
            
            refs.append(row["ref"].values[0])
            
            alts.append(row["alt"].values[0])

            rows[i] = row.drop(["ref","alt"], axis=1).reset_index(drop=True)
            
    if not len(set(refs)) == 1 and len(set(alts)) == 1:
        
        raise ValueError(pos, rows)
        
    ref = refs[0]
    
    alt = alts[0]

    mean_row = np.sum(rows[i] for i in rows.keys()) / len(rows.keys())
    
    mean_row["count"] = len(rows.keys())
    
    mean_row["ref"] = ref
    
    mean_row["alt"] = alt
    
    assert mean_row["position"].values[0] == pos, row
    
    #print(pos, mean_row)
    
    gam_c_average = gam_c_average.append(mean_row)

##### 2Rd

##### for identifying concordant SNPs in 2Rd, we use the entire inversion

In [ ]:
sites_2Rd = '( (POS > 31495381) & (POS < 42375004) )'

filter_2Rd = v_2R.eval(sites_2Rd)

np.sum(filter_2Rd)

In [ ]:
g_2Rd = g_2R.subset(sel1 = md_2R["ox_code"].isin(splits["2Rd"]["train"]).values)

md_2Rd = md_2R.loc[md_2R["ox_code"].isin(splits["2Rd"]["train"]),:]

In [ ]:
d_dict = {}

for i in range(10):
    print(i)
    
    train, test =\
    model_selection.train_test_split(md_2Rd["ox_code"].values, train_size=0.75, test_size=0.25,
                                 stratify=md_2Rd["new_PCA_2Rd"].values)
    
    train_bool = md_2Rd["ox_code"].isin(train).values
    test_bool = md_2Rd["ox_code"].isin(test).values
    
    d = ingenos.run_concordance_calculation("2Rd", v_2R[:], g_2Rd, 
                                md_2Rd.loc[train_bool, "new_PCA_2Rd"].map(float).values,
                                        variance_threshold = 0.05, sites_bool = filter_2Rd,
                                samples_bool = train_bool)
    
    top = d.loc[((d["called_0"] > 0.9) & (d["called_1"] > 0.9) & (d["called_2"] > 0.9) &\
       (d["min"] > 0.8)),"position"].values
    
    site_indices = np.array([np.where(v_2R["POS"] == site)[0][0] for site in top])

    alts = g_2Rd.subset(sel0 = site_indices, sel1 = test_bool).to_n_alt()
    
    assigned = []

    for specimen in np.mean(alts, axis=0):
        
        if specimen <= 0.66:
        
            assigned.append(0)
        
        elif specimen > 0.66 and specimen <= 1.33:

            assigned.append(1)

        else:

            assigned.append(2)
            
    md_2Rd_test = pd.DataFrame(md_2Rd.loc[test_bool,:])
    
    md_2Rd_test["assigned"] = pd.Series(assigned).values
    
    mismatches =\
    np.sum(md_2Rd_test["assigned"].map(float).map(str) != md_2Rd_test["new_PCA_2Rd"])
    
    d_dict[i] = (train, test, d, top, mismatches)

In [ ]:
d_compiled = []

for i in range(10):
    
    d_compiled.extend(d_dict[i][2]["position"].values)

In [ ]:
d_average = pd.DataFrame({"position" : sorted(set(d_compiled)),
                          "ref" : np.nan,
                          "alt" : np.nan,
                         "score_0" : np.nan, 
                          "score_1" : np.nan, 
                          "score_2" : np.nan,
                         "overall_score" : np.nan, 
                          "called_0" : np.nan, 
                          "called_1" : np.nan,
                         "called_2" : np.nan, 
                          "overall_called" : np.nan, 
                          "min" : np.nan})\
[["position","ref","alt","score_0","score_1","score_2","overall_score","called_0","called_1",
 "called_2","overall_called","min"]]

In [ ]:
for pos in sorted(set(d_compiled)):
    
    rows = {}
    
    refs = []
    
    alts = []

    for i in range(10):

        row = d_dict[i][2][d_dict[i][2]["position"] == pos]

        if len(row) > 0:
            
            refs.append(row["ref"].values[0])
            
            alts.append(row["alt"].values[0])

            rows[i] = row.drop(["ref","alt"], axis=1).reset_index(drop=True)
            
    if not len(set(refs)) == 1 and len(set(alts)) == 1:
        
        raise ValueError(pos, rows)
        
    ref = refs[0]
    
    alt = alts[0]

    mean_row = np.sum(rows[i] for i in rows.keys()) / len(rows.keys())
    
    mean_row["count"] = len(rows.keys())
    
    mean_row["ref"] = ref
    
    mean_row["alt"] = alt
    
    assert mean_row["position"].values[0] == pos, row
        
    d_average = d_average.append(mean_row)

##### 2Ru

In [ ]:
sites_2Ru = ingenos.construct_filter_expression("2Ru", ingenos.inversionDict,
                                                buffer=0, whole_inversion=True)

filter_2Ru = v_2R.eval(sites_2Ru)

np.sum(filter_2Ru)

In [ ]:
g_2Ru = g_2R.subset(sel1 = md_2R["ox_code"].isin(splits["2Ru"]["train"]).values)

md_2Ru = md_2R.loc[md_2R["ox_code"].isin(splits["2Ru"]["train"]),:]

In [ ]:
u_dict = {}

for i in range(10):
    print(i)
    
    train, test =\
    model_selection.train_test_split(md_2Ru["ox_code"].values, train_size=0.75, test_size=0.25,
                                 stratify=md_2Ru["new_PCA_2Ru"].values)
    
    train_bool = md_2Ru["ox_code"].isin(train).values
    test_bool = md_2Ru["ox_code"].isin(test).values
    
    u = ingenos.run_concordance_calculation("2Ru", v_2R[:], g_2Ru, 
                                md_2Ru.loc[train_bool, "new_PCA_2Ru"].map(float).values,
                                        variance_threshold = 0.05, sites_bool = filter_2Ru,
                                samples_bool = train_bool)
    
    top = u.loc[((u["called_0"] > 0.9) & (u["called_1"] > 0.9) & (u["called_2"] > 0.9) &\
       (u["min"] > 0.8)),"position"].values
    
    site_indices = np.array([np.where(v_2R["POS"] == site)[0][0] for site in top])

    alts = g_2Ru.subset(sel0 = site_indices, sel1 = test_bool).to_n_alt()
    
    assigned = []

    for specimen in np.mean(alts, axis=0):
        
        if specimen <= 0.66:
        
            assigned.append(0)
        
        elif specimen > 0.66 and specimen <= 1.33:

            assigned.append(1)

        else:

            assigned.append(2)
            
    md_2Ru_test = pd.DataFrame(md_2Ru.loc[test_bool,:])
    
    md_2Ru_test["assigned"] = pd.Series(assigned).values
    
    mismatches =\
    np.sum(md_2Ru_test["assigned"].map(str) != md_2Ru_test["new_PCA_2Ru"])
    
    u_dict[i] = (train, test, u, top, mismatches)

In [ ]:
u_compiled = []

for i in range(10):
    
    u_compiled.extend(u_dict[i][2]["position"].values)

In [ ]:
u_average = pd.DataFrame({"position" : sorted(set(u_compiled)),
                          "ref" : np.nan,
                          "alt" : np.nan,
                         "score_0" : np.nan, 
                          "score_1" : np.nan, 
                          "score_2" : np.nan,
                         "overall_score" : np.nan, 
                          "called_0" : np.nan, 
                          "called_1" : np.nan,
                         "called_2" : np.nan, 
                          "overall_called" : np.nan, 
                          "min" : np.nan})\
[["position","ref","alt","score_0","score_1","score_2","overall_score","called_0","called_1",
 "called_2","overall_called","min"]]

In [ ]:
for pos in sorted(set(u_compiled)):
    
    rows = {}
    
    refs = []
    
    alts = []

    for i in range(10):

        row = u_dict[i][2][u_dict[i][2]["position"] == pos]

        if len(row) > 0:
            
            refs.append(row["ref"].values[0])
            
            alts.append(row["alt"].values[0])

            rows[i] = row.drop(["ref","alt"], axis=1).reset_index(drop=True)
            
    if not len(set(refs)) == 1 and len(set(alts)) == 1:
        
        raise ValueError(pos, rows)
        
    ref = refs[0]
    
    alt = alts[0]

    mean_row = np.sum(rows[i] for i in rows.keys()) / len(rows.keys())
    
    mean_row["count"] = len(rows.keys())
    
    mean_row["ref"] = ref
    
    mean_row["alt"] = alt
    
    assert mean_row["position"].values[0] == pos, row
        
    u_average = u_average.append(mean_row)

##### take the top SNPs for each inversion

In [ ]:
a_top = a_average.loc[((a_average["called_0"] > 0.9) & (a_average["called_1"] > 0.9) &\
                       (a_average["called_2"] > 0.9) &\
               (a_average["min"] > 0.995) & (a_average["count"] >= 8)), "position"].map(int)

In [ ]:
j_top = j_average.loc[((j_average["called_0"] > 0.9) & (j_average["called_1"] > 0.9) &\
                       (j_average["called_2"] > 0.9) &\
               (j_average["min"] > 0.8) & (j_average["count"] >= 8)), "position"].map(int)

In [ ]:
b_top = b_average.loc[((b_average["called_0"] > 0.9) & (b_average["called_1"] > 0.9) &\
                       (b_average["called_2"] > 0.9) &\
               (b_average["min"] > 0.8) & (b_average["count"] >= 8)), "position"].map(int)

In [ ]:
c_col_top = col_average.loc[((col_average["called_0"] > 0.9) &\
                             (col_average["called_1"] > 0.9) &\
                         (col_average["called_2"] > 0.9) &\
               (col_average["min"] > 0.8) & (col_average["count"] >= 8)), "position"].map(int)

In [ ]:
c_gam_top = gam_average.loc[((gam_average["called_0"] > 0.9) &\
                             (gam_average["called_1"] > 0.9) &\
                         (gam_average["called_2"] > 0.9) &\
               (gam_average["min"] > 0.8) & (gam_average["count"] >= 8)), "position"].map(int)

In [ ]:
d_top = d_average.loc[((d_average["called_0"] > 0.9) & (d_average["called_1"] > 0.9) &\
                       (d_average["called_2"] > 0.9) &\
               (d_average["min"] > 0.8) & (d_average["count"] >= 8)), "position"].map(int)

In [ ]:
u_top = u_average.loc[((u_average["called_0"] > 0.9) & (u_average["called_1"] > 0.9) &\
                       (u_average["called_2"] > 0.9) &\
               (u_average["min"] > 0.8) & (u_average["count"] >= 8)), "position"].map(int)

## testing in validation set

##### subset the data and the metadata

In [ ]:
g_2La = g_2L.subset(sel1 = md_2L["ox_code"].isin(splits["2La"]["test"]).values)
g_2Rj = g_2R.subset(sel1 = md_2R["ox_code"].isin(splits["2Rj"]["test"]).values)
g_2Rb = g_2R.subset(sel1 = md_2R["ox_code"].isin(splits["2Rb"]["test"]).values)
g_2Rc = g_2R.subset(sel1 = md_2R["ox_code"].isin(splits["2Rc"]["test"]).values)
g_2Rd = g_2R.subset(sel1 = md_2R["ox_code"].isin(splits["2Rd"]["test"]).values)
g_2Ru = g_2R.subset(sel1 = md_2R["ox_code"].isin(splits["2Ru"]["test"]).values)

md_2La = md_2L.loc[md_2L["ox_code"].isin(splits["2La"]["test"]),:]
md_2Rb = md_2R.loc[md_2R["ox_code"].isin(splits["2Rb"]["test"]),:]
md_2Rc = md_2R.loc[md_2R["ox_code"].isin(splits["2Rc"]["test"]),:]
md_2Rd = md_2R.loc[md_2R["ox_code"].isin(splits["2Rd"]["test"]),:]
md_2Rj = md_2R.loc[md_2R["ox_code"].isin(splits["2Rj"]["test"]),:]
md_2Ru = md_2R.loc[md_2R["ox_code"].isin(splits["2Ru"]["test"]),:]

##### then, subset by species for 2Rc. we also drop the Bamako specimens (Bamako is a chromosomal form defined by 2Rjcu == 2, with 2Rb polymorphic; in our data set, all gambiae specimens with 2Rj == 2 are Bamako, so we can use that as a shortcut); Bamako appears to be on an independent evolutionary trajectory

In [ ]:
g_2Rc = g_2Rc.subset(sel1 = md_2Rc["2Rj"] != "2.0")
md_2Rc = md_2Rc.loc[md_2Rc["2Rj"] != "2.0",:]

col_bool = (md_2Rc["species"] == "An. coluzzii").values
gam_bool = (md_2Rc["species"] == "An. gambiae").values

g_2Rc_col = g_2Rc.subset(sel1 = col_bool)
g_2Rc_gam = g_2Rc.subset(sel1 = gam_bool)

col_md = md_2Rc.loc[col_bool,:]
gam_md = md_2Rc.loc[gam_bool,:]

In [ ]:
Inversion = namedtuple('Inversion',['SNPs','metadata','genotypes','inv_title'])

In [ ]:
inv_dict = {"2La" : Inversion(SNPs = a_top.values, metadata = md_2La, genotypes = g_2La,
                             inv_title = "new_PCA_2La"),
           "2Rb" : Inversion(SNPs = b_top.values, metadata = md_2Rb, genotypes = g_2Rb,
                            inv_title = "new_PCA_2Rb"),
            "2Rc_col" : Inversion(SNPs = c_col_top.values, metadata = col_md, 
                                 genotypes = g_2Rc_col,
                             inv_title = "new_PCA_2Rc"),
           "2Rc_gam" : Inversion(SNPs = c_gam_top.values, metadata = gam_md, 
                                genotypes = g_2Rc_gam,
                            inv_title = "new_PCA_2Rc")
           "2Rd" : Inversion(SNPs = d_top.values, metadata = md_2Rd, genotypes = g_2Rd,
                            inv_title = "new_PCA_2Rd"),
           "2Rj" : Inversion(SNPs = j_top.values, metadata = md_2Rj, genotypes = g_2Rj,
                            inv_title = "new_PCA_2Rj"),
           "2Ru" : Inversion(SNPs = u_top.values, metadata = md_2Ru, genotypes = g_2Ru,
                            inv_title = "new_PCA_2Ru")}

In [ ]:
for inversion in inv_dict.keys():
    
    ##set up objects
    SNPs = inv_dict[inversion].SNPs
    md = inv_dict[inversion].metadata
    gt = inv_dict[inversion].genotypes[:]
    col_name = inv_dict[inversion].inv_title
    new_col_name = inversion + "_assigned"
    mean_name = inversion + "_means"
    
    if inversion == "2La":
        
        vt = v_2L[:]
        
    else:
        
        vt = v_2R[:]
    
    ##identify sites found in the data
    site_indices = []
    
    for site in SNPs:
    
        where = np.where(vt["POS"] == site)
        
        if len(where[0]) > 0:
                
            site_indices.append(where[0][0])
            
    print(inversion, "# targets: ", str(len(SNPs)), " # found: ", str(len(site_indices)))
    
    ##identify biallelic sites
    
    bi_bool = gt.subset(sel0 = site_indices).count_alleles().max_allele() <= 1
        
    alts = gt.subset(sel0 = site_indices).subset(sel0 = bi_bool).to_n_alt()
        
    is_called = gt.subset(sel0 = site_indices).subset(sel0 = bi_bool).is_called()
    
    av_gts = np.mean(np.ma.MaskedArray(
            alts, mask = ~is_called), axis=0).data
            
    total_sites = np.sum(is_called, axis=0)
        
    karyos = []
    
    for alt in av_gts:
        
        if alt <= (2/3):
            
            karyos.append(0)
            
        elif alt > (2/3) and alt <= (4/3):
            
            karyos.append(1)
            
        else:
            
            karyos.append(2)
            
    md[new_col_name] = karyos
    md[mean_name] = av_gts
    
    mismatches = np.sum(md[new_col_name] != md[col_name].map(float).map(int))
    
    print(inversion, " # mismatches: ", mismatches,"\n")
    print(av_gts)
    print(total_sites,"\n")

for inversion in inv_dict.keys():
    
    ##set up objects
    SNPs = inv_dict[inversion].SNPs
    md = inv_dict[inversion].metadata
    gt = inv_dict[inversion].genotypes[:]
    col_name = inv_dict[inversion].inv_title
    new_col_name = inversion + "_assigned"
    mean_name = inversion + "_means"
    called_name = inversion + "_called"
    match_name = inversion + "_sites_matching"
    match_proportion_name = inversion + "_pct_sites_matching"

    if inversion == "2La":
        
        vt = v_2L[:]
        
    else:
        
        vt = v_2R[:]
    
    ##identify sites found in the data
    site_indices = []
    
    for site in SNPs:
    
        where = np.where(vt["POS"] == site)
        
        if len(where[0]) > 0:
                
            site_indices.append(where[0][0])
            
    print(inversion, "# targets: ", str(len(SNPs)), " # found: ", str(len(site_indices)))
    
    ##identify biallelic sites
    
    bi_bool = gt.subset(sel0 = site_indices).count_alleles().max_allele() <= 1
        
    alts = gt.subset(sel0 = site_indices).subset(sel0 = bi_bool).to_n_alt()
        
    is_called = gt.subset(sel0 = site_indices).subset(sel0 = bi_bool).is_called()
    
    av_gts = np.mean(np.ma.MaskedArray(
            alts, mask = ~is_called), axis=0).data
    
    match_dict = {0: None, 1: None, 2: None}
    
    for value in [0,1,2]:
    
        n_matches = np.sum(np.ma.MaskedArray(alts, mask = ~is_called) == value, axis=0)
        match_dict[value] = n_matches
            
    total_sites = np.sum(is_called, axis=0)
        
    karyos = []
    
    for alt in av_gts:
        
        if alt <= (2/3):
            
            karyos.append(0)
            
        elif alt > (2/3) and alt <= (4/3):
            
            karyos.append(1)
            
        else:
            
            karyos.append(2)
            
    match_list = []
    
    for index, karyo in enumerate(karyos):
        
        match_list.append(match_dict[karyo][index])
            
    md[new_col_name] = karyos
    md[mean_name] = av_gts
    md[called_name] = total_sites
    md[match_name] = pd.Series(match_list)
    md[match_proportion_name] = md[match_name] / md[called_name]
    
    print(inversion,"\n")
    print(av_gts)
    print(total_sites,"\n")

##### repeat for all specimens to karyotype them all

In [ ]:
inv_dict = {"2La" : Inversion(SNPs = a_top.values, metadata = md_2L, genotypes = g_2L,
                             inv_title = "new_PCA_2La"),
            "2Rj" : Inversion(SNPs = j_top.values, metadata = md_2R, genotypes = g_2R,
                            inv_title = "new_PCA_2Rj"),
           "2Rb" : Inversion(SNPs = b_top.values, metadata = md_2R, genotypes = g_2R,
                            inv_title = "new_PCA_2Rb"),
            "2Rc_col" : Inversion(SNPs = c_col_top.values, metadata = md_2R, genotypes = g_2R,
                            inv_title = "new_PCA_2Rc"),
           "2Rc_gam" : Inversion(SNPs = c_gam_top.values, metadata = md_2R, genotypes = g_2R, 
                            inv_title = "new_PCA_2Rc"),
           "2Rd" : Inversion(SNPs = d_top.values, metadata = md_2R, genotypes = g_2R,
                            inv_title = "new_PCA_2Rd"),
           "2Ru" : Inversion(SNPs = u_top.values, metadata = md_2R, genotypes = g_2R,
                            inv_title = "new_PCA_2Ru")}

In [ ]:
for inversion in inv_dict.keys():
    
    ##set up objects
    SNPs = inv_dict[inversion].SNPs
    md = inv_dict[inversion].metadata
    gt = inv_dict[inversion].genotypes[:]
    col_name = inv_dict[inversion].inv_title
    new_col_name = inversion + "_assigned"
    mean_name = inversion + "_means"
    called_name = inversion + "_called"
    match_name = inversion + "_sites_matching"
    match_proportion_name = inversion + "_pct_sites_matching"

    if inversion == "2La":
        
        vt = v_2L[:]
        
    else:
        
        vt = v_2R[:]
    
    ##identify sites found in the data
    site_indices = []
    
    for site in SNPs:
    
        where = np.where(vt["POS"] == site)
        
        if len(where[0]) > 0:
                
            site_indices.append(where[0][0])
            
    print(inversion, "# targets: ", str(len(SNPs)), " # found: ", str(len(site_indices)))
    
    ##identify biallelic sites
    
    bi_bool = gt.subset(sel0 = site_indices).count_alleles().max_allele() <= 1
        
    alts = gt.subset(sel0 = site_indices).subset(sel0 = bi_bool).to_n_alt()
        
    is_called = gt.subset(sel0 = site_indices).subset(sel0 = bi_bool).is_called()
    
    av_gts = np.mean(np.ma.MaskedArray(
            alts, mask = ~is_called), axis=0).data
    
    match_dict = {0: None, 1: None, 2: None}
    
    for value in [0,1,2]:
    
        n_matches = np.sum(np.ma.MaskedArray(alts, mask = ~is_called) == value, axis=0)
        match_dict[value] = n_matches
            
    total_sites = np.sum(is_called, axis=0)
        
    karyos = []
    
    for alt in av_gts:
        
        if alt <= (2/3):
            
            karyos.append(0)
            
        elif alt > (2/3) and alt <= (4/3):
            
            karyos.append(1)
            
        else:
            
            karyos.append(2)
            
    match_list = []
    
    for index, karyo in enumerate(karyos):
        
        match_list.append(match_dict[karyo][index])
            
    md[new_col_name] = karyos
    md[mean_name] = av_gts
    md[called_name] = total_sites
    md[match_name] = pd.Series(match_list)
    md[match_proportion_name] = md[match_name] / md[called_name]
    
    print(inversion,"\n")
    print(av_gts)
    print(total_sites,"\n")

## testing in low coverage

##### read in and prepare data from Main et al.

##### karyotypes taken from Main et al. 2015 supplemental file Molecular Ecology 2015 Main.xlsx, tab "Fig1_genotype_data"; VCF from Dryad [give doi]

##### read in and prepare metadata

In [ ]:
Main_good_names = ["02SEL85","04SEL02","04SEL14","04SEL021","04SEL18","04SEL84","04SEL91",
                   "010sel134","O10SEL160",
               "2012SEL002","2012SEL003","2012SEL006","2012SEL009","2012sel012",
                   "2012SEL013","2012sel029","2012sel063"]

Main_sample_bool = [sample in Main_good_names for sample in Main_2R["samples"]]

Main_a = [2,2,1,1,2,2,2,2,2,1,2,2,2,2,2,2,2]
Main_b = [2,2,2,1,2,0,0,0,1,2,0,2,2,0,2,1,1]
Main_c = [2,2,0,1,2,0,0,0,1,0,0,2,2,0,2,1,1]
Main_d = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
Main_j = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
Main_u = [0,0,0,0,0,0,2,2,1,0,0,0,0,1,0,0,0]

Main_md = pd.DataFrame({"sample_ID" : pd.Series(Main_2R["samples"][Main_sample_bool]),
                       "2La" : pd.Series(Main_a),
                       "2Rb" : pd.Series(Main_b),
                       "2Rc" : pd.Series(Main_c),
                       "2Rd" : pd.Series(Main_d),
                       "2Rj" : pd.Series(Main_j),
                       "2Ru" : pd.Series(Main_u)})

Main_md = Main_md[["sample_ID","2La","2Rb","2Rc","2Rd","2Rj","2Ru"]]


##### read in data, filter, and mask low-quality genotypes

In [ ]:
Main_2L = allel.read_vcf(
    '/afs/crc.nd.edu/group/BesanskyNGS/data05/comp_karyo/data/Agam_normfilt_vcfs.vcf.gz',
                   fields = ['*'], region = "2L", types={'calldata/GQ': 'f4'})

Main_2R = allel.read_vcf(
    '/afs/crc.nd.edu/group/BesanskyNGS/data05/comp_karyo/data/Agam_normfilt_vcfs.vcf.gz',
                   fields = ['*'], region = "2R", types={'calldata/GQ': 'f4'})


Main_2R_gt = allel.GenotypeArray(Main_2R["calldata/GT"]).subset(sel1 = Main_sample_bool)

Main_2R_gq = Main_2R["calldata/GQ"][:,Main_sample_bool]

Main_2R_gt.mask = Main_2R_gq < 20

Main_2L_gt = allel.GenotypeArray(Main_2L["calldata/GT"]).subset(sel1 = Main_sample_bool)

Main_2L_gq = Main_2L["calldata/GQ"][:,Main_sample_bool]

Main_2L_gt.mask = Main_2L_gq < 20

In [2]:
Main_inv_dict = {"2La" : Inversion(SNPs = a_top.values, metadata = Main_md, 
                                   genotypes = Main_2L_gt, inv_title = "2La"),
                 "2Rj" : Inversion(SNPs = j_top.values, metadata = Main_md, 
                             genotypes = Main_2R_gt, inv_title = "2Rj"),
           "2Rb" : Inversion(SNPs = b_top.values, metadata = Main_md, 
                             genotypes = Main_2R_gt, inv_title = "2Rb"),
        "2Rc_col" : Inversion(SNPs = col_top.values, metadata = Main_md, 
                               genotypes = Main_2R_gt, inv_title = "2Rc"),
           "2Rc_gam" : Inversion(SNPs = gam_top.values, metadata = Main_md, 
                                  genotypes = Main_2R_gt, inv_title = "2Rc"),
           "2Rd" : Inversion(SNPs = d_top.values, metadata = Main_md, 
                             genotypes = Main_2R_gt, inv_title = "2Rd"),
           "2Ru" : Inversion(SNPs = u_top.values, metadata = Main_md, 
                             genotypes = Main_2R_gt, inv_title = "2Ru")}

NameError: name 'namedtuple' is not defined

In [ ]:
for inversion in Main_inv_dict.keys():
    
    ##set up objects
    SNPs = Main_inv_dict[inversion].SNPs
    md = Main_inv_dict[inversion].metadata
    gt = Main_inv_dict[inversion].genotypes
    col_name = Main_inv_dict[inversion].inv_title
    new_col_name = inversion + "_assigned"
    mean_name = inversion + "_means"
    sites_name = inversion + "_sites_called"
    match_name = inversion + "_sites_matching"
    match_proportion_name = inversion + "_pct_sites_matching"
    
    if inversion == "2La":
        
        pos = Main_2L["variants/POS"]
        
    else:
        
        pos = Main_2R["variants/POS"]
    
    ##identify sites found in the data
    site_indices = []
    
    for site in SNPs:
    
        where = np.where(pos == site)
        
        if len(where[0]) > 0:
                
            site_indices.append(where[0][0])
            
    print(inversion, "# targets: ", str(len(SNPs)), " # found: ", str(len(site_indices)))
    
    ##identify biallelic sites
    
    bi_bool = gt.subset(sel0 = site_indices).count_alleles().max_allele() <= 1
        
    alts = gt.subset(sel0 = site_indices).subset(sel0 = bi_bool).to_n_alt()
        
    is_called = gt.subset(sel0 = site_indices).subset(sel0 = bi_bool).is_called()
    
    av_gts = np.mean(np.ma.MaskedArray(
            alts, mask = ~is_called), axis=0).data
    
    match_dict = {0: None, 1: None, 2: None}
    
    for value in [0,1,2]:
    
        n_matches = np.sum(np.ma.MaskedArray(alts, mask = ~is_called) == value, axis=0)
        match_dict[value] = n_matches
    
    total_sites = np.sum(is_called, axis=0)
        
    karyos = []
    
    for alt in av_gts:
        
        if alt <= (2/3):
            
            karyos.append(0)
            
        elif alt > (2/3) and alt <= (4/3):
            
            karyos.append(1)
            
        else:
            
            karyos.append(2)
    
    match_list = []
    
    for index, karyo in enumerate(karyos):
        
        match_list.append(match_dict[karyo][index])
            
    md[new_col_name] = pd.Series(karyos)
    md[mean_name] = pd.Series(av_gts)
    md[sites_name] = pd.Series(total_sites)
    md[match_name] = pd.Series(match_list)
    md[match_proportion_name] = md[match_name] / md[sites_name]
    
    mismatches = np.sum(md[new_col_name] != md[col_name])
    
    print(inversion, " # mismatches: ", mismatches,"\n")
    print(av_gts)
    print(total_sites,"\n")

##### repeat with the Love et al. specimens. data from [doi], metadata from [file]

##### assemble the metadata

In [ ]:
Love_good_names = ['KL0218','KL0220','KL0231','KL0333','KL0341','KL0370','KL0671','KL0899']

Love_a = [2,2,2,np.nan,np.nan,np.nan,np.nan,np.nan]
Love_b = [2,0,0,0,0,2,2,0]
Love_c = [2,2,2,2,2,2,2,2]
Love_d = [0,0,0,0,0,0,0,0]
Love_j = [2,2,2,2,2,2,2,2]
Love_u = [2,2,2,2,2,2,2,2]

Love_md = pd.DataFrame({"sample_ID" : pd.Series(Love_good_names),
                       "2La" : pd.Series(Love_a),
                       "2Rb" : pd.Series(Love_b),
                       "2Rc" : pd.Series(Love_c),
                       "2Rd" : pd.Series(Love_d),
                       "2Rj" : pd.Series(Love_j),
                       "2Ru" : pd.Series(Love_u)})

Love_md = Love_md[["sample_ID","2La","2Rb","2Rc","2Rd","2Rj","2Ru"]]

##### read in the data, filter it, and mask low-quality genotypes

In [ ]:
Love_2R = allel.read_vcf(
    '/afs/crc.nd.edu/group/BesanskyNGS/data02/16G_bamako/all.Bamakoset.2.recode.vcf.gz',
                   fields = ['*'], region = "2R", types={'calldata/GQ': 'f4'})

Love_2L = allel.read_vcf(
    '/afs/crc.nd.edu/group/BesanskyNGS/data02/16G_bamako/all.Bamakoset.2.recode.vcf.gz',
                   fields = ['*'], region = "2L", types={'calldata/GQ': 'f4'})

Love_sample_bool = [sample in Love_good_names for sample in Love_2R["samples"]]

Love_2R_gt = allel.GenotypeArray(Love_2R["calldata/GT"]).subset(sel1 = Love_sample_bool)
Love_2R_gq = Love_2R["calldata/GQ"][:,Love_sample_bool]

Love_2R_gt.mask = Love_2R_gq < 20

Love_2L_gt = allel.GenotypeArray(Love_2L["calldata/GT"]).subset(sel1 = Love_sample_bool)

Love_2L_gq = Love_2L["calldata/GQ"][:,Love_sample_bool]

Love_2L_gt.mask = Love_2L_gq < 20

In [ ]:
Love_inv_dict = {"2La" : Inversion(SNPs = a_top.values, metadata = Love_md, 
                                   genotypes = Love_2L_gt, inv_title = "2La"),
           "2Rb" : Inversion(SNPs = b_top.values, metadata = Love_md, 
                             genotypes = Love_2R_gt, inv_title = "2Rb"),
            "2Rc_col" : Inversion(SNPs = col_top.values, metadata = Love_md, 
                                   genotypes = Love_2R_gt, inv_title = "2Rc"),
           "2Rc_gam" : Inversion(SNPs = gam_top.values, metadata = Love_md, 
                                  genotypes = Love_2R_gt, inv_title = "2Rc"),
 "2Rd" : Inversion(SNPs = d_top.values, metadata = Love_md, 
                             genotypes = Love_2R_gt, inv_title = "2Rd"),
           "2Rj" : Inversion(SNPs = j_top.values, metadata = Love_md, 
                             genotypes = Love_2R_gt, inv_title = "2Rj"),
           "2Ru" : Inversion(SNPs = u_top.values, metadata = Love_md, 
                             genotypes = Love_2R_gt, inv_title = "2Ru")}

In [3]:
for inversion in Love_inv_dict.keys():
    
    ##set up objects
    SNPs = Love_inv_dict[inversion].SNPs
    md = Love_inv_dict[inversion].metadata
    gt = Love_inv_dict[inversion].genotypes
    col_name = Love_inv_dict[inversion].inv_title
    new_col_name = inversion + "_assigned"
    mean_name = inversion + "_means"
    sites_name = inversion + "_sites_called"
    match_name = inversion + "_sites_matching"
    match_proportion_name = inversion + "_pct_sites_matching"
    
    if inversion == "2La":
        
        pos = Love_2L["variants/POS"]
        
    else:
        
        pos = Love_2R["variants/POS"]
    
    ##identify sites found in the data
    site_indices = []
    
    for site in SNPs:
    
        where = np.where(pos == site)
        
        if len(where[0]) > 0:
                
            site_indices.append(where[0][0])
            
    print(inversion, "# targets: ", str(len(SNPs)), " # found: ", str(len(site_indices)))
    
    ##identify biallelic sites
    
    bi_bool = gt.subset(sel0 = site_indices).count_alleles().max_allele() <= 1
        
    alts = gt.subset(sel0 = site_indices).subset(sel0 = bi_bool).to_n_alt()
        
    is_called = gt.subset(sel0 = site_indices).subset(sel0 = bi_bool).is_called()
    
    av_gts = np.mean(np.ma.MaskedArray(
            alts, mask = ~is_called), axis=0).data
    
    match_dict = {0: None, 1: None, 2: None}
    
    for value in [0,1,2]:
    
        n_matches = np.sum(np.ma.MaskedArray(alts, mask = ~is_called) == value, axis=0)
        match_dict[value] = n_matches
    
    total_sites = np.sum(is_called, axis=0)
        
    karyos = []
    
    for alt in av_gts:
        
        if alt <= (2/3):
            
            karyos.append(0)
            
        elif alt > (2/3) and alt <= (4/3):
            
            karyos.append(1)
            
        else:
            
            karyos.append(2)
    
    match_list = []
    
    for index, karyo in enumerate(karyos):
        
        match_list.append(match_dict[karyo][index])
            
    md[new_col_name] = pd.Series(karyos)
    md[mean_name] = pd.Series(av_gts)
    md[sites_name] = pd.Series(total_sites)
    md[match_name] = pd.Series(match_list)
    md[match_proportion_name] = md[match_name] / md[sites_name]
    
    mismatches = np.sum(md[new_col_name] != md[col_name])
    
    print(inversion, " # mismatches: ", mismatches,"\n")
    print(av_gts)
    print(total_sites,"\n")

NameError: name 'Love_inv_dict' is not defined